<a href="https://colab.research.google.com/github/samirjhb/samirjhb/blob/main/clase06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir streaming

In [8]:
%%python --bg
import time
import pandas as pd
import numpy as np



def aleatorio():
  return int(99*np.random.normal(0, 0.1)+1)

for n in range(0,200):
  pd.DataFrame(np.asarray([aleatorio()for i in range(0,100)])\
               .reshape(-1, 1), columns=['dato'])\
               .to_csv('/content/streaming/data'+str(n)+'.csv', index=False)

  time.sleep(5)

In [2]:
%%capture
!pip install pyspark
!pip install pyarrow

In [28]:
from  pyspark.sql import  SparkSession
spark=SparkSession.builder\
      .master('local')\
      .appName('ejemplo')\
      .config('spark.ui.port',4050).getOrCreate()

from pyspark.sql.types import *
schema=StructType([StructField('dato', IntegerType(), True)])
df= spark.readStream\
   .format('csv').schema(schema)\
   .option('header', True).load('streaming')

In [15]:
## Verificar si esta funcionado
df.isStreaming

True

In [27]:
## Detener el spark
spark.stop()

In [29]:
## Procesamiento de los cambio  en memoria
query = df.writeStream.outputMode('update')\
          .queryName('tabla')\
          .format('memory')\
          .start()

In [33]:
from os import truncate

## Monitoreo
spark.sql("select mean(dato) from tabla").show(truncate=False)

+----------+
|mean(dato)|
+----------+
|0.94525   |
+----------+



In [36]:
## Procesamiento de los cambio  subir archivo
result_df=df.select('*')
query = result_df.writeStream.format('json')\
          .option('checkpointLocation', 'checkpoint')\
          .option('path', 'results')\
          .queryName('tabla')\
          .start()

In [35]:
## Detener el query
query.stop()